In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lreg").getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [4]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [5]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol='prediction')

In [6]:
lrModel = lr.fit(training)

In [7]:
lrModel.coefficients

Out[68]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

Out[69]: 0.14228558260358093

In [9]:
training_summary = lrModel.summary

In [10]:
training_summary.r2

Out[71]: 0.027839179518600154

In [11]:
training_summary.rootMeanSquaredError

Out[72]: 10.16309157133015

In [12]:
# Doing train test split

In [13]:
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [14]:
split_object = all_data.randomSplit([0.7, 0.3])

In [15]:
split_object

Out[76]: [DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [16]:
train_data, test_data =  all_data.randomSplit([0.7, 0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 337|
 mean| 0.8902631621797588|
 stddev| 10.093917046302407|
 min|-26.736207182601724|
 max| 27.78383192005107|
+-------+-------------------+

In [18]:
test_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 164|
 mean|-1.0446182057005393|
 stddev| 10.677159931739245|
 min|-28.571478869743427|
 max| 26.903524792043335|
+-------+-------------------+

In [19]:
correct_model = lr.fit(train_data)

In [20]:
test_results = correct_model.evaluate(test_data)

In [21]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 -27.35569207326169|
 -27.94453645559413|
 -28.418279593185|
-23.621108789638143|
-28.025831762456036|
-19.770660266077755|
-22.571856902511726|
 -21.93890422166436|
 -20.67534761227699|
 -16.36652317913169|
 -18.13739480279236|
-19.742107820477976|
-17.733742687378943|
-16.051156773066133|
 -17.54614467392917|
-12.921944165361426|
-16.161783593119385|
-16.031266557361867|
-13.340481199384339|
 -17.33552225587857|
+-------------------+
only showing top 20 rows

In [22]:
test_results.rootMeanSquaredError

Out[83]: 10.789021410705892

In [23]:
unlabeled_data = test_data.select('features')

In [24]:
test_data.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
-28.571478869743427|(10,[0,1,2,3,4,5,...|
-28.046018037776633|(10,[0,1,2,3,4,5,...|
-26.805483428483072|(10,[0,1,2,3,4,5,...|
-23.487440120936512|(10,[0,1,2,3,4,5,...|
-22.949825936196074|(10,[0,1,2,3,4,5,...|
-22.837460416919342|(10,[0,1,2,3,4,5,...|
-21.432387764165806|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
 -19.66731861537172|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
-17.026492264209548|(10,[0,1,2,3,4,5,...|
-16.692207021311106|(10,[0,1,2,3,4,5,...|
-16.151349351277112|(10,[0,1,2,3,4,5,...|
 -16.08565904102149|(10,[0,1,2,3,4,5,...|
-15.334767479922341|(10,[0,1,2,3,4,5,...|
-15.310980589416289|(10,[0,1,2,3,4,5,...|
-15.056482974542433|(10,[0,1,2,3,4,5,...|
-14.822152909751189|(10,[0,1,2,3,4,5,...|
-13.867087895158768|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [25]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [26]:
predictions = correct_model.transform(unlabeled_data) # Real world scenario where we don't have the labels and can't get measures of goodness of the model

In [27]:
predictions.show()

+--------------------+-------------------+
 features| prediction|
+--------------------+-------------------+
(10,[0,1,2,3,4,5,...|-1.2157867964817373|
(10,[0,1,2,3,4,5,...|-0.1014815821825037|
(10,[0,1,2,3,4,5,...| 1.6127961647019313|
(10,[0,1,2,3,4,5,...|0.13366866870163152|
(10,[0,1,2,3,4,5,...| 5.076005826259962|
(10,[0,1,2,3,4,5,...| -3.066800150841588|
(10,[0,1,2,3,4,5,...| 1.1394691383459215|
(10,[0,1,2,3,4,5,...| 1.8814216058751474|
(10,[0,1,2,3,4,5,...| 1.0080289969052703|
(10,[0,1,2,3,4,5,...| -2.80176944383207|
(10,[0,1,2,3,4,5,...| 1.110902538582812|
(10,[0,1,2,3,4,5,...| 3.04990079916687|
(10,[0,1,2,3,4,5,...| 1.58239333610183|
(10,[0,1,2,3,4,5,...|-0.0345022679553586|
(10,[0,1,2,3,4,5,...| 2.2113771940068285|
(10,[0,1,2,3,4,5,...|-2.3890364240548627|
(10,[0,1,2,3,4,5,...| 1.10530061857695|
(10,[0,1,2,3,4,5,...| 1.2091136476106799|
(10,[0,1,2,3,4,5,...|-0.5266066957744291|
(10,[0,1,2,3,4,5,...| 3.5630806941756967|
+--------------------+-------------------+
only showing top 20 rows